# Slimming down cleaning process and make it able to take in any new data
##### Started 14.12.22

### Imports

In [16]:
import pandas as pd
import random
import string
import re
import json
import ast
from collections import Counter


### Getting workable DF

In [2]:
data_path = '~/code/emilycardwell/final-project-data/data/raw/kaggle_raw.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(3)

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber


In [3]:
slim_raw_df = raw_df[['artist_name', 'song_name', 'chords', 'genres', 'popularity']]
slim_raw_df.head(3)

,artist_name,song_name,chords,genres,popularity
0,Justin Bieber,"10,000 Hours",{3: 'G G/B ...,"['canadian pop', 'pop', 'post-teen pop']",100
1,Justin Bieber,2 Much,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","['canadian pop', 'pop', 'post-teen pop']",100
2,Justin Bieber,2u (feat. David Guetta),{1: 'Em D C C...,"['canadian pop', 'pop', 'post-teen pop']",100


In [132]:
split_raw_df = slim_raw_df.copy()
splits = []

for row in split_raw_df.chords:
    row_dict = ast.literal_eval(row)
    for key, val in row_dict.items():
        p1 = re.sub(r'\||\s-|-[^/]|,|\.|\*', ' ', val)
        p2 = re.sub(r'[()]', '', p1)
        s = re.sub(r'^\s+|\s+$', '', p2)
        row_dict[key] = re.sub(r'\s+', ' ', s)
    splits.append(row_dict)

split_raw_df.chords = splits
len(splits)

135783

In [133]:
#r = random.randint(0, 135782)
print(slim_raw_df.chords[r])
print(split_raw_df.chords[r])

{0: '\n\t  \t\tIntroduçao: Bm2 | G | Bm2 | Bm2 | Bm2 | G | Em7 | F#m | Bm2 | G | Em7 | A... ', 3: '           Bm7                  A ', 5: '             D/F#             G  ', 7: '       Bm7               A ', 9: '         A/G    G ', 11: '               Bm7             A ', 13: '                 D/F#               G ', 15: '        Bm7             A ', 17: '            A/G      G        A   Bm ', 20: ' A     D/F#  Em7          D/F# ', 22: '        G             A ', 24: ' Em7             D/F#            G           (A) ', 29: '          D       D/F# ', 31: '           E/G#       A ', 33: '             Em  Bm7         A ', 39: 'G Bm7 A               ', 41: 'G Bm7 A               ', 43: 'G Bm7 A               ', 45: 'Em7  G   A ', 48: 'G Bm7 A               ', 50: 'G Bm7 A               ', 52: 'G Bm7 A               ', 54: 'Em7  G   A ', 59: 'D      D/F#  E/G#    Asus4 A        D/F#  G     A                     D '}
{0: 'Introduçao: Bm2 G Bm2 Bm2 Bm2 G Em7 F#m Bm2 G Em7 A', 3: 'Bm7 A', 

In [131]:
r

125933

#### See Chord Frequency

In [53]:
chords_counter = Counter()
for song in splits:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)
del chords_counter['']

chord_count_df = pd.Series(dict(chords_counter)).to_frame('chord_count')
sorted_cc_df = chord_count_df.sort_values(by='chord_count', ascending=False)
len(sorted_cc_df)

29750

In [57]:
letters = list(string.ascii_uppercase)[:7]
non_chord_filter = [chord for chord in sorted_cc_df.index if chord[0] in letters]
cc_df = sorted_cc_df[sorted_cc_df.index.isin(non_chord_filter)]

# pd.set_option('display.max_rows', 1190)
## sorted_cc_df.head(70*17)
pd.set_option('display.max_rows', 100)
cc_df.iloc[0:100]

,chord_count
E7/B,478
A5+,478
Gb7M,478
Gm/D,474
Dbmaj7,474
B7sus4,470
Adim,467
Break:,464
A7sus,463
Ab9,463


In [ ]:
# slim_chord_counts_dict = {}
# for chord, count in chords_count_dict.items():
#     if count <= low_freq_to_remove:
#         pass
#     else:
#         slim_chord_counts_dict[chord] = count

## Creating List of Correctly Formatted Chords

In [59]:
major_chords = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']

In [60]:
chords_df = pd.DataFrame(index=['m', 'dim', 'aug', '5', 'aug5', '5b7',
                                'dim7', 'hdim7', 'm7', 'm-M7', '7', 'aug7', 'M7',  
                                'm9', '9', '7b9',  'M9', 'm11', '11',
                                'add2', 'add4', 'add6', 'add9', 
                                'madd2', 'madd4', 'madd6', 'madd9', 
                                '7add4', '7addb6', '7add6', 
                                'msus2', 'msus4', '7sus2', '7sus4', 'sus2', 'sus4'
                                ],
                         columns=major_chords)


In [61]:
for idx in chords_df.index:
    new_chords = [m + idx for m in major_chords]
    chords_df.loc[idx, :] = new_chords
chords_df.head(3)

,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
m,Cm,C#m,Dbm,Dm,D#m,Ebm,Em,Fm,F#m,Gbm,Gm,G#m,Abm,Am,A#m,Bbm,Bm
dim,Cdim,C#dim,Dbdim,Ddim,D#dim,Ebdim,Edim,Fdim,F#dim,Gbdim,Gdim,G#dim,Abdim,Adim,A#dim,Bbdim,Bdim
aug,Caug,C#aug,Dbaug,Daug,D#aug,Ebaug,Eaug,Faug,F#aug,Gbaug,Gaug,G#aug,Abaug,Aaug,A#aug,Bbaug,Baug


In [70]:
sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 
                'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
flats_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 
                'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
sharp_keys = ['D', 'E', 'G', 'A', 'B', 'Dm', 'Em', 'Fm', 'Bm']
flats_keys = ['C', 'F', 'Cm', 'Gm', 'Am']

slash_indexes = ['dim/b7', 'm/2', 'm/b3', 'm/3', 'm/4', 'm/5', 'm/6','m/b7', 'm/7',
                'm7/b3', 'm7/4', 'm7/b5', 'm7/5', 'm7/7', 
                '/2', '/b3', '/3', '/4', '/5', '/6','/b7', '/7', 
                '7/b2', '7/2', '7/b3', '7/3', '7/4', '7/5', '7/b6', '7/6','7/7', 
                '9/b3', '9/3', '9/4', '9/5', '9/6', '9/7',
                'M7/2', 'M7/b3', 'M7/3', 'M7/4', 'M7/5', 'M7/6','M7/b7']

whole_to_half = {'b2': 1, '2': 2, 'b3': 3, '3': 4, '4': 5, 'b5': 6,
                 '5': 7, 'b6': 8,'6': 9, 'b7': 10, '7': 11}

In [76]:
# big function
def get_slash_notes(slash_indexes, columns):
    slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
    i = 0 
    
    def find_slash(chord, notes, i):
        for si in slash_indexes:
            s = si.split('/')
            h = whole_to_half[s[1]]
            sc = f'{chord}{s[0]}/{notes[i+h]}'
            slash_chord.loc[si, chord] = sc
    
    def sort_f_s(chord, notes):
        for n in notes:
            if n in chord :
                i = notes.index(chord[0])
                find_slash(chord, notes, i)
            elif len(chord) > 1:
                if '#' == chord[1]:
                    i = sharp_notes.index(chord[:2])
                    find_slash(chord, sharp_notes, i)
                elif 'b' == chord[1]:
                    i = flats_notes.index(chord[:2])
                    find_slash(chord, flats_notes, i)
    
    for chord in columns:
        sort_f_s(chord, sharp_notes)
        sort_f_s(chord, flats_notes)
                
            
    return slash_chord

In [103]:
slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
slash_chords_df

,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
m,Cm,C#m,Dbm,Dm,D#m,Ebm,Em,Fm,F#m,Gbm,Gm,G#m,Abm,Am,A#m,Bbm,Bm
dim,Cdim,C#dim,Dbdim,Ddim,D#dim,Ebdim,Edim,Fdim,F#dim,Gbdim,Gdim,G#dim,Abdim,Adim,A#dim,Bbdim,Bdim
aug,Caug,C#aug,Dbaug,Daug,D#aug,Ebaug,Eaug,Faug,F#aug,Gbaug,Gaug,G#aug,Abaug,Aaug,A#aug,Bbaug,Baug
5,C5,C#5,Db5,D5,D#5,Eb5,E5,F5,F#5,Gb5,G5,G#5,Ab5,A5,A#5,Bb5,B5
aug5,Caug5,C#aug5,Dbaug5,Daug5,D#aug5,Ebaug5,Eaug5,Faug5,F#aug5,Gbaug5,Gaug5,G#aug5,Abaug5,Aaug5,A#aug5,Bbaug5,Baug5
5b7,C5b7,C#5b7,Db5b7,D5b7,D#5b7,Eb5b7,E5b7,F5b7,F#5b7,Gb5b7,G5b7,G#5b7,Ab5b7,A5b7,A#5b7,Bb5b7,B5b7
dim7,Cdim7,C#dim7,Dbdim7,Ddim7,D#dim7,Ebdim7,Edim7,Fdim7,F#dim7,Gbdim7,Gdim7,G#dim7,Abdim7,Adim7,A#dim7,Bbdim7,Bdim7
hdim7,Chdim7,C#hdim7,Dbhdim7,Dhdim7,D#hdim7,Ebhdim7,Ehdim7,Fhdim7,F#hdim7,Gbhdim7,Ghdim7,G#hdim7,Abhdim7,Ahdim7,A#hdim7,Bbhdim7,Bhdim7
m7,Cm7,C#m7,Dbm7,Dm7,D#m7,Ebm7,Em7,Fm7,F#m7,Gbm7,Gm7,G#m7,Abm7,Am7,A#m7,Bbm7,Bm7
m-M7,Cm-M7,C#m-M7,Dbm-M7,Dm-M7,D#m-M7,Ebm-M7,Em-M7,Fm-M7,F#m-M7,Gbm-M7,Gm-M7,G#m-M7,Abm-M7,Am-M7,A#m-M7,Bbm-M7,Bm-M7


In [80]:
80 * 17

1360

## Shove chords into correct format

#### For next time:
- merge chords
- delete everything other than those

In [107]:
substitutions = {'': 'maj', '': ',', 'dim': 'º', 'aug': '5+', 'M7': ['7M', 'maj7'], 
                 'hdim7': 'm7b5', 'm-M7': 'm7+', 'aug7': '7+', '9': '79', 
                 'madd2': r'^2', 'madd4': r'^4', 'madd6': r'^6', 
                 'add2': r'^2', 'add4': r'^4', 'add6': r'^6', 'sus4': r'sus$',
                 '7add4': '711', '7addb6': '7b13', '7add6': '713', '9add4': '4/7/9', '9add6': '69', 
                 'dim/b7': 'm5-/7', 
                 '/b2': '/9-', '/2': '/9', '/b5': '/5-', '/b6': ['/13-', '/5+'], '/6': '/13'
                 }

In [115]:
rejects = {}

def merge_chords(chords_column):
    merged_chords_column = chords_column.copy()
    
    for song in merged_chords_column:
        for measure, chords in song.items():
            ch_list = chords.split()
            for idx, ch in enumerate(ch_list):
                if ch == '%':
                    ch.replace('%', ch_list[idx-1])
                elif ch in substitutions.values():
                    ch.replace(ch, list(filter(lambda x: ch in substitutions[x], substitutions))[0])
                elif ch in slash_chords_df:
                    pass  
                else:
                    if ch in rejects.keys():
                        rejects[ch] += 1
                    else:
                        rejects[ch] = 1
                    merged_chords_column.drop
    
    return merged_chords_column

In [121]:
merge_chords(split_raw_df.chords)

0         {3: 'G G/B C', 5: 'G', 7: 'G G/B C', 9: 'G', 1...
1         {1: 'Intro: F#m7 D2', 4: 'F#m7', 8: 'D2', 10: ...
2         {1: 'Em D C C D Em', 3: 'Em D C C D Em', 5: 'E...
3         {4: 'Intro: Em Bm Am C 2x', 6: 'Em Bm', 8: 'Am...
4         {2: 'Intro: Gm - Dm - C - C x2', 5: 'Gm Dm', 7...
                                ...                        
135778    {0: 'B E', 2: 'B E', 4: 'B E-F#-E-B-F# E', 6: ...
135779    {2: 'Intro: A59', 5: 'A59 G', 7: 'Am', 9: 'C F...
135780    {2: 'Dm C', 4: 'Dm', 6: 'F Bb F', 8: 'Bb F', 1...
135781    {3: 'G Em', 5: 'C G', 7: 'G Am', 9: 'C D', 11:...
135782    {0: 'Intro: D F G F D F G F D F G F D F G F', ...
Name: chords, Length: 135783, dtype: object

In [122]:
print('Rejects:', {c: q for c, q in rejects.items() if q > 100})

Rejects: {'G/B': 76514, 'Em': 1080717, 'Bm': 668989, 'Intro:': 112799, 'F#m7': 72989, 'D2': 4973, 'A/C#': 29692, 'Am': 1203719, 'D/F#': 83714, '2x': 23903, 'Gm': 278039, '-': 91385, 'Dm': 565824, 'x2': 22542, 'Intro': 44657, '2x:': 788, 'Cm': 207123, 'C/E': 26873, 'INTRO:': 7502, 'Am,': 2834, 'Dm,': 1449, 'E,': 2243, 'F#m': 441055, 'C#m': 286533, 'Em7': 191956, 'Dm7': 124710, 'Am7': 201609, 'Bm7': 127498, 'Cadd9': 56088, 'break': 1178, 'C/G': 31253, 'G#m': 148244, 'Interlude': 6025, 'play': 365, 'once': 522, 'E/G': 774, 'E/B': 4087, 'Bbmaj7': 5722, 'Bb7': 23086, 'Ebm': 40043, 'C7': 91475, 'D#m': 56965, 'F7M': 27372, 'C7M': 33052, 'X': 759, '2': 5755, '3': 1160, 'Outro:': 2990, 'Eb7M': 6745, 'C9': 65783, 'B7': 179446, 'C7M/G': 537, 'Cm6': 3645, 'F#4/D': 140, 'G9': 17986, 'Dm6': 11846, 'Bridge:': 21857, '4x': 7292, 'Em9': 6779, 'D4': 12923, 'hide': 5573, 'this': 5737, 'tab': 5629, 'Final': 3433, 'Bridge': 27398, 'intro:': 1797, 'F/A': 17101, 'G6/E': 332, 'Final:': 2660, 'Dsus4': 20619, '

In [ ]:
# big function
def translate_slashes(slash_indexes, columns):
    slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
    i = 0 
    
    def find_slash(chord, notes, i):
        for si in slash_indexes:
            s = si.split('/')
            h = whole_to_half[s[1]]
            sc = f'{chord}{s[0]}/{notes[i+h]}'
            slash_chord.loc[si, chord] = sc
    
    def sort_f_s(chord, notes):
        for n in notes:
            if n in chord :
                i = notes.index(chord[0])
                find_slash(chord, notes, i)
            elif len(chord) > 1:
                if '#' == chord[1]:
                    i = sharp_notes.index(chord[:2])
                    find_slash(chord, sharp_notes, i)
                elif 'b' == chord[1]:
                    i = flats_notes.index(chord[:2])
                    find_slash(chord, flats_notes, i)
    
    for chord in columns:
        sort_f_s(chord, sharp_notes)
        sort_f_s(chord, flats_notes)
                
            
    return slash_chord

In [ ]:
slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
